In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



In [23]:
url = "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_special,video&release_date=2025-01-01,2025-10-30&interests=in0000112"


In [ ]:
## Irei coletar o nome de 100 filmes do IMDB
try:
    driver.get(url)
    print("Página carregada. Aguardando conteúdo...")
    time.sleep(3)
    
    # Clicar no botão "Mais 50" usando JavaScript
    try:
        botao = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='ipc-see-more__text']"))
        )
        # Usar JavaScript para clicar (evita problemas de overlay)
        driver.execute_script("arguments[0].click();", botao)
        print(" Clicou em 'Mais 50' via JavaScript. Aguardando...")
        time.sleep(5)
    except Exception as e:
        print(f" Erro ao clicar: {e}")
    
    # Coletar filmes
    soup = bs(driver.page_source, 'html.parser')
    filmes = soup.find_all('h3', class_='ipc-title__text')
    
    todos_filmes = []
    for filme in filmes:
        texto = filme.text.strip()
        if '. ' in texto:
            titulo = texto.split('. ', 1)[1]
            todos_filmes.append(titulo)
    
    todos_filmes = todos_filmes[:100]
    print(f"\n Total: {len(todos_filmes)}")
    
finally:
    driver.quit()


Página carregada. Aguardando conteúdo...
✅ Clicou em 'Mais 50' via JavaScript. Aguardando...

✅ Total: 100
Primeiros 5: ['A Hora do Mal', 'A Longa Marcha - Caminhe ou Morra', 'Frankenstein', 'O Telefone Preto 2', 'A Mão que Balança o Berço']
Últimos 5: ['Screamboat - Terror a Bordo', 'Stay', 'Su from So', 'Drácula', 'A Vingança de Popeye']


In [ ]:
## Aqui irei coletar as notas dos 100 filmes
try:
    driver.get(url)
    print("Página carregada. Coletando notas...")
    time.sleep(3)
    
    # Clicar no botão "Mais 50" usando JavaScript
    try:
        botao = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='ipc-see-more__text']"))
        )
        driver.execute_script("arguments[0].click();", botao)
        print(" Clicou em 'Mais 50'. Aguardando...")
        time.sleep(5)
    except Exception as e:
        print(f" Erro ao clicar: {e}")
    
    # Coletar notas
    soup = bs(driver.page_source, 'html.parser')
    notas = soup.find_all('span', class_='ipc-rating-star--rating')
    
    todas_notas = []
    for nota in notas:
        todas_notas.append(nota.text.strip())
    
    todas_notas = todas_notas[:100]  # Limitar a 100
    print(f"\n Total de notas: {len(todas_notas)}")
    
finally:
    driver.quit()

In [ ]:
# Por fim irei coletar as URLs de cada filme para poder obter as reviews de cada um deles.
try:
    driver.get(url)
    print("Página carregada. Coletando links...")
    time.sleep(3)
    
    # Clicar no botão "Mais 50" usando JavaScript
    try:
        botao = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='ipc-see-more__text']"))
        )
        driver.execute_script("arguments[0].click();", botao)
        time.sleep(5)
    except Exception as e:
        print(f"⚠️ Erro ao clicar: {e}")
    
    # Coletar links
    soup = bs(driver.page_source, 'html.parser')
    links = soup.find_all('a', class_='ipc-title-link-wrapper')
    
    urls_filmes = []
    for link in links:
        href = link.get('href')
        if href and '/title/' in href:
            url_completa = f"https://www.imdb.com{href}"
            urls_filmes.append(url_completa)
    
    urls_filmes = urls_filmes[:100]  # Limitar a 100
    print(f"\n Total de URLs: {len(urls_filmes)}")
    
finally:
    driver.quit()



In [96]:
# Ajustar para o mesmo tamanho (preencher com None onde não há nota)
tamanho_maximo = max(len(todos_filmes), len(todas_notas), len(urls_filmes))
tamanho_minimo = min(len(todos_filmes), len(todas_notas), len(urls_filmes))

# Usar o tamanho dos filmes como referência (100)
df_filmes = pd.DataFrame({
    'Titulo': todos_filmes[:100],
    'Link': urls_filmes[:100],
    'Nota_IMDb': (todas_notas + [None] * 100)[:100]  # Preenche com None se faltar
})


In [98]:
def coletar_reviews(url_filme, num_reviews=20):
    """Coleta reviews clicando no botão 'Ver tudo'"""
    url_base = url_filme.replace('/pt/title/', '/title/')
    imdb_id = url_base.split("/title/")[1].split("/")[0].split("?")[0]
    url_reviews = f"https://www.imdb.com/title/{imdb_id}/reviews/"
    
    stop_phrases = [
        'Featured in', 'Written and Performed by', 'Courtesy of', 'Official Trailer'
    ]
    
    try:
        
        driver.get(url_reviews)
        time.sleep(3)
        
        # Scroll até o botão aparecer
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        
        try:
            # Procurar o botão que contém "Ver tudo" para ver todas as reviews
            botao = driver.find_element(By.XPATH, "//button[.//span[text()='Ver tudo']]")
            driver.execute_script("arguments[0].scrollIntoView();", botao)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", botao)
            time.sleep(5)
        except Exception as e:
            print(f" Botão não encontrado: {e}")
        
        # Coletar reviews
        soup = bs(driver.page_source, 'html.parser')
        reviews_elements = soup.find_all('div', class_='ipc-html-content-inner-div')
        
        reviews = []
        for review in reviews_elements:
            texto = review.text.strip()
            contem_stop = any(stop in texto for stop in stop_phrases)
            
            if len(texto) > 50 and not contem_stop:
                reviews.append(texto)
                
                if len(reviews) >= num_reviews:
                    break
        
        return reviews[:num_reviews]
        
    except Exception as e:
        print(f" Erro: {e}")
        return []


In [ ]:
# Percebi que a análise geral da primeira review era um sentimento negativo. Mas na verdade, a review era extremamente positiva!
# Com isso em mente resolvi verificar se palavras fortemente ligadas ao terror estavam sendo mal interpretadas:
palavras_teste = [
    "scary and terrifying",  # Deveria ser positivo para terror
    "boring and bad",  # Deveria ser negativo
    "suspenseful and creepy",  # Deveria ser positivo para terror
    "amazing horror film",  # Positivo
    "disappointing and slow"  # Negativo
]

print("TESTANDO PALAVRAS:")
for frase in palavras_teste:
    score = analyzer.polarity_scores(frase)
    print(f"\n'{frase}'")
    print(f"Compound: {score['compound']}")

TESTANDO PALAVRAS:

'scary and terrifying'
Compound: -0.7845

'boring and bad'
Compound: -0.7003

'suspenseful and creepy'
Compound: 0.0

'amazing horror film'
Compound: 0.0258

'disappointing and slow'
Compound: -0.4939


## Problemática encontrada:
Como observado acima, em um contexto de filmes de terror palavras como 'assustador' deveriam ser interpretadas como indicadores positivos, mas estavam sendo interpretados como negativo ao invés disso. Com isso conclui que é necessário atualizar o léxico do Vader, adicionando palavras fortemente ligadas ao gênero e seus respectivos pesos. Após realizar o ajuste, notei que o compound ligado a palavras de terror melhor significativemente. Dito isso, irei prosseguir com todas as reviews de todos os filmes.


In [ ]:
## função para identificar sentimentos nas reviews
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Adicionar palavras de terror como POSITIVAS
palavras_terror_positivas = {
    'scary': 2.5,
    'terrifying': 3.0,
    'horrifying': 3.0,
    'creepy': 2.5,
    'disturbing': 2.5,
    'suspenseful': 2.5,
    'tense': 2.0,
    'atmospheric': 2.0,
    'eerie': 2.5,
    'chilling': 2.5,
    'unsettling': 2.5,
    'haunting': 2.5,
    'dark': 1.5,
    'twisted': 2.0,
    'shocking': 2.0,
    'intense': 2.5,
    'thrilling': 3.0,
    'gripping': 2.5,
    'spine-chilling': 3.5,
    'frightening': 3.0,
    'gore': 2.0,
    'gory': 2.0,
    'bloody': 1.5,
    'brutal': 2.0,
    'horror': 2.0
}

# Atualizar o léxico do VADER
analyzer.lexicon.update(palavras_terror_positivas)

def analisar_sentimento_review(texto):
    """Analisa sentimento de uma review"""
    scores = analyzer.polarity_scores(texto)
    
    return {
        'compound': scores['compound'],  
        'positive': scores['pos'],
        'negative': scores['neg'],
        'neutral': scores['neu']
    }

def analisar_sentimento_filme_terror(reviews):
    """Analisa reviews de filme de terror com léxico ajustado"""
    if not reviews:
        return None
    
    sentimentos = []
    for review in reviews:
        score = analyzer.polarity_scores(review)
        sentimentos.append(score['compound'])
    
    return {
        'sentimento_medio': sum(sentimentos) / len(sentimentos),
        'sentimento_max': max(sentimentos),
        'sentimento_min': min(sentimentos),
        'num_reviews': len(reviews),
        'reviews_positivas': sum(1 for s in sentimentos if s >= 0.05),
        'reviews_negativas': sum(1 for s in sentimentos if s <= -0.05)
    }

In [ ]:
# Coloquei aqui algumas palavras-chave que considero boas para categorizar emoções em filmes de terror
emocoes_terror = {
    'medo_intenso': ['terrifying', 'scary', 'frightening', 'horrifying', 'creepy', 'disturbing'],
    'suspense': ['suspenseful', 'tense', 'thrilling', 'gripping', 'edge of seat'],
    'gore': ['bloody', 'gore', 'gruesome', 'violent', 'graphic'],
    'atmosfera': ['atmospheric', 'eerie', 'haunting', 'dark', 'unsettling'],
    'jump_scares': ['jump scare', 'startling', 'shocking']
}

def classificar_emocoes_terror(reviews):

    contagem = {emocao: 0 for emocao in emocoes_terror.keys()}
    
    for review in reviews:
        review_lower = review.lower()
        for emocao, palavras in emocoes_terror.items():
            if any(palavra in review_lower for palavra in palavras):
                contagem[emocao] += 1
    
    return contagem

In [100]:
from tqdm import tqdm
import time

# Lista para armazenar resultados completos
resultados_completos = []

for idx, row in tqdm(df_filmes.iterrows(), total=len(df_filmes), desc="Coletando reviews"):
    try:
        print(f"\n{idx+1}. {row['Titulo']}")
        
        # Coletar reviews
        reviews = coletar_reviews(row['Link'], num_reviews=20)
        
        if reviews:
            # Análise de sentimento geral
            sentimentos = analisar_sentimento_filme_terror(reviews)
            
            # Classificar emoções específicas de terror
            emocoes = classificar_emocoes_terror(reviews)
            
            resultado = {
                'Titulo': row['Titulo'],
                'Nota_IMDb': row['Nota_IMDb'],
                'Link': row['Link'],
                **sentimentos,
                **emocoes
            }
            
            resultados_completos.append(resultado)
            print(f"{len(reviews)} reviews | Sentimento: {sentimentos['sentimento_medio']:.3f}")
        else:
            print("Nenhuma review encontrada")
        
        time.sleep(2)
        
    except Exception as e:
        print(f"Erro: {e}")
        continue


df_analise = pd.DataFrame(resultados_completos)

print(df_analise.head())

Coletando reviews:   0%|          | 0/100 [00:00<?, ?it/s]


1. A Hora do Mal
Acessando: https://www.imdb.com/title/tt26581740/reviews/
  ✅ Clicou em 'Ver tudo'
20 reviews | Sentimento: 0.818


Coletando reviews:   1%|          | 1/100 [00:18<29:43, 18.01s/it]


2. A Longa Marcha - Caminhe ou Morra
Acessando: https://www.imdb.com/title/tt10374610/reviews/
  ✅ Clicou em 'Ver tudo'
20 reviews | Sentimento: 0.436


Coletando reviews:   2%|▏         | 2/100 [00:34<27:42, 16.96s/it]

...


99. Drácula
Acessando: https://www.imdb.com/title/tt32448239/reviews/
  ⚠️ Botão não encontrado: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[.//span[text()='Ver tudo']]"}
  (Session info: chrome=141.0.7390.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x110fe43+66515]
	GetHandleVerifier [0x0x110fe84+66580]
	(No symbol) [0x0xefdc48]
	(No symbol) [0x0xf48704]
	(No symbol) [0x0xf48aab]
	(No symbol) [0x0xf8f482]
	(No symbol) [0x0xf6b214]
	(No symbol) [0x0xf8cba7]
	(No symbol) [0x0xf6afc6]
	(No symbol) [0x0xf3c2ca]
	(No symbol) [0x0xf3d154]
	GetHandleVerifier [0x0x1367353+2521315]
	GetHandleVerifier [0x0x13622d3+2500707]
	GetHandleVerifier [0x0x1137c94+229924]
	GetHandleVerifier [0x0x11281f8+165768]
	GetHandleVerifier [0x0x112ecad+193085]
	GetHandleVerifier [0x0x1118158+100072]
	GetHandleVerifier [0x0x11182f0+

Coletando reviews:  99%|█████████▉| 99/100 [21:02<00:11, 11.35s/it]


100. A Vingança de Popeye
Acessando: https://www.imdb.com/title/tt33362807/reviews/
  ✅ Clicou em 'Ver tudo'
20 reviews | Sentimento: -0.044


Coletando reviews: 100%|██████████| 100/100 [21:17<00:00, 12.78s/it]

                              Titulo Nota_IMDb  \
0                      A Hora do Mal       7,5   
1  A Longa Marcha - Caminhe ou Morra       6,9   
2                       Frankenstein       7,8   
3                 O Telefone Preto 2       6,6   
4          A Mão que Balança o Berço       5,3   

                                                Link  sentimento_medio  \
0  https://www.imdb.com/pt/title/tt26581740/?ref_...          0.817640   
1  https://www.imdb.com/pt/title/tt10374610/?ref_...          0.436060   
2  https://www.imdb.com/pt/title/tt1312221/?ref_=...          0.799455   
3  https://www.imdb.com/pt/title/tt29644189/?ref_...          0.516095   
4  https://www.imdb.com/pt/title/tt33502160/?ref_...          0.340175   

   sentimento_max  sentimento_min  num_reviews  reviews_positivas  \
0          0.9957         -0.9509           20                 19   
1          0.9964         -0.9525           20                 14   
2          0.9993         -0.9253           20 

In [101]:
df_analise

,Titulo,Nota_IMDb,Link,sentimento_medio,sentimento_max,sentimento_min,num_reviews,reviews_positivas,reviews_negativas,medo_intenso,suspense,gore,atmosfera,jump_scares
0,A Hora do Mal,"7,5",https://www.imdb.com/pt/title/tt26581740/?ref_...,0.817640,0.9957,-0.9509,20,19,1,7,6,1,7,1
1,A Longa Marcha - Caminhe ou Morra,"6,9",https://www.imdb.com/pt/title/tt10374610/?ref_...,0.436060,0.9964,-0.9525,20,14,6,3,4,3,3,1
2,Frankenstein,"7,8",https://www.imdb.com/pt/title/tt1312221/?ref_=...,0.799455,0.9993,-0.9253,20,19,1,2,0,2,5,0
3,O Telefone Preto 2,"6,6",https://www.imdb.com/pt/title/tt29644189/?ref_...,0.516095,0.9986,-0.9773,20,15,4,11,4,4,6,1
4,A Mão que Balança o Berço,"5,3",https://www.imdb.com/pt/title/tt33502160/?ref_...,0.340175,0.9961,-0.9210,20,14,6,0,3,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Screamboat - Terror a Bordo,"3,2",https://www.imdb.com/pt/title/tt30766582/?ref_...,0.317175,0.9977,-0.9782,20,12,7,3,0,9,3,1
90,Stay,"7,7",https://www.imdb.com/pt/title/tt38039379/?ref_...,0.623536,0.9951,-0.9286,14,12,2,2,3,2,4,1
91,Su from So,"6,0",https://www.imdb.com/pt/title/tt37629310/?ref_...,0.855490,0.9980,0.2924,20,20,0,1,3,0,0,0
92,Drácula,"3,2",https://www.imdb.com/pt/title/tt32448239/?ref_...,0.902467,0.9596,0.8650,3,3,0,0,0,0,0,1


In [ ]:
# Aqui eu calculo o score do 'terror efetivo'
score_emocoes = (
    (df_analise['medo_intenso'] / df_analise['num_reviews']) * 4 +
    (df_analise['suspense'] / df_analise['num_reviews']) * 2 +
    (df_analise['atmosfera'] / df_analise['num_reviews']) * 2 +
    (df_analise['gore'] / df_analise['num_reviews']) * 1 +
    (df_analise['jump_scares'] / df_analise['num_reviews']) * 1
)

# Penaliza filmes com poucas views para não desbalancear as notas
sentimento_normalizado = (df_analise['sentimento_medio'] + 1) / 2

# a nota final é composta por 50% do score de terror e 50% dos sentimentos obtidos
score_bruto = (score_emocoes * 0.5) + (sentimento_normalizado * 10 * 0.5)

# Fator de confiança
min_reviews = 20
df_analise['fator_confianca'] = df_analise['num_reviews'].apply(lambda x: min(x / min_reviews, 1.0))

# Score final
df_analise['score_terror'] = score_bruto * df_analise['fator_confianca']

# Ranking
ranking_terror = df_analise.sort_values('score_terror', ascending=False)

print("\n TOP 10 FILMES DE TERROR MAIS ASSUSTADORES:\n")
print(ranking_terror[['Titulo', 'Nota_IMDb', 'score_terror', 'sentimento_medio', 'num_reviews', 'medo_intenso']].head(10))



 TOP 10 FILMES DE TERROR MAIS ASSUSTADORES:

                           Titulo Nota_IMDb  score_terror  sentimento_medio  \
11                Faça Ela Voltar       7,2      6.702587          0.691035   
86                        Rosario       7,7      6.023688          0.629475   
23               A Meia-Irmã Feia       7,0      6.017275          0.936910   
0                   A Hora do Mal       7,5      5.944100          0.817640   
31              Animais Perigosos       6,4      5.806062          0.782425   
6                      Bom Menino       6,3      5.787662          0.875065   
18                         Juntos       6,7      5.784825          0.813930   
22  Premonição 6: Laços de Sangue       6,7      5.719288          0.927715   
3              O Telefone Preto 2       6,6      5.515237          0.516095   
19    Until Dawn: Noite de Terror       5,7      5.467625          0.827050   

    num_reviews  medo_intenso  
11           20            14  
86           20     